# Configuration

> Define the data classes using Pydantic, making it possible to configure the chat application and do input validation.

In [ ]:
#| default_exp config

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
#| hide
from pydantic import BaseModel, Field
from typing import Optional, List, Tuple, Literal, Any
import os
import gradio as gr
from pathlib import Path
from dotenv import load_dotenv

## Some lessons and clarification about the used code

1. `Pydantic` vs `Dataclasses`: Pydantic creates classes similar to Python's dataclasses but with additional features. The key differences are that Pydantic provides data validation, type coercion, and more robust error handling. It will automatically validate data during initialization and conversion.
2. Pydantic and typing: Pydantic leverages Python's standard typing system but adds its own validation on top. It uses Python's type hints to know what types to validate against.
3. The "..." placeholder: The ellipsis (...) is a special value in Pydantic that indicates a required field. It means "this field must be provided when creating an instance" - there's no default value. When you create a ModelConfig instance, you'll need to provide a value for model_name.
4. `@property` usage: The `@property` decorator creates a getter method that's accessed like an attribute. In our case, api_key looks like a normal attribute but when accessed, it runs the method to retrieve the value from environment variables. This is a clean way to avoid storing sensitive information in the object itself.
5. `Field` from `pydantic` can be used to add extra information and metadata to inform the reader and/or do data validation.

## The configuration for the workings of the LLM chatbot

#### ModelConfig

First the configuration for the LLM model to use in the `ModelConfig`.

In [ ]:
#| export
class ModelConfig(BaseModel):
    """Configuration for the LLM model"""
    model_name: str = Field(..., description="Name or path of the model to use")
    provider: str = Field(default="huggingface", description="Model provider (huggingface, openai, etc)")
    api_key_env_var: Optional[str] = Field(default=None, description="Environment variable name for API key")
    api_base_url: Optional[str] = Field(default=None, description="Base URL for API reqeuest")
    temperature: float = Field(default=0.7, description="Temperature for generation")
    max_completion_tokens: int = Field(default=1024, description="Maximum tokens to generate")
    top_p: float = Field(default=0.7, description="Adjust the number of choices for each predicted token [0-1]")
    top_k: int = Field(default=50, description="Limits the number of choices for the next predicted token. Not available for OpenAI API")
    frequency_penalty: float = Field(default=0, description="Reduces the likelihood of repeating prompt text or getting stuck in a loop [-2 -> 2]")
    stop: Optional[List[str]] = Field(default=["\nUser:", "<|endoftext|>"], description="Sequences to stop generation")
    stream: Optional[bool] = Field(default=None, description="If set to true, the model response data will be streamed to the client as it is generated using server-sent events.")

    
    @property
    def api_key(self) -> Optional[str]:
        """Get the API key from environment variables if specified"""
        if self.api_key_env_var:
            if os.environ.get(self.api_key_env_var):
                return os.environ.get(self.api_key_env_var)
            raise ValueError(f"The environment variable {self.api_key_env_var} is not found in the .env file.")
        return None

#### Message config

Next the configuration of the Message system

In [ ]:
#| export
class Message(BaseModel):
    """A message in a conversation"""
    role: Literal["system", "user", "assistant"] = Field(..., description="Role of the message sender")
    content: str = Field(..., description="Content of the message")

#### Chat config

Then the configuration for the chat implementation. Making sure the application can handle:
- system prompt
- context if applicable
- a start 'user' prompt if applicable
- user input

The other settings that are available in this class can easily be infered from the description in the `ChatAppConfig` class itself.

In [ ]:
#| export
class ChatAppConfig(BaseModel):
    """Main configuration for a chat application"""
    app_name: str = Field(..., description="Name of the application")
    description: str = Field(default="", description="Description of the application")
    system_prompt: str = Field(..., description="System prompt for the LLM")
    starter_prompt: Optional[str] = Field(default=None, description="Initial prompt to start the conversation")
    context_files: List[Path] = Field(default=[], description="List of markdown files for additional context")
    model: ModelConfig
    theme: Optional[Any] = Field(default=None, description="Gradio theme to use")
    logo_path: Optional[Path] = Field(default=None, description="Path to logo image")
    show_system_prompt: bool = Field(default=True, description="Whether to show system prompt in UI")
    show_context: bool = Field(default=True, description="Whether to show context in UI")

An example configuration for a chat application:

In [ ]:
test_config = ChatAppConfig(
    app_name="Test App",
    system_prompt="You are a helpful assistant.",
    model=ModelConfig(
        model_name="gpt-3.5-turbo",
        api_key_env_var="HF_API_KEY",
    )
)

print(test_config.model_dump_json(indent=2))

print(f"API Key available: {'Yes' if test_config.model.api_key else 'No'}")

{
  "app_name": "Test App",
  "description": "",
  "system_prompt": "You are a helpful assistant.",
  "starter_prompt": null,
  "context_files": [],
  "model": {
    "model_name": "gpt-3.5-turbo",
    "provider": "huggingface",
    "api_key_env_var": "HF_API_KEY",
    "api_base_url": null,
    "max_tokens": 1024,
    "temperature": 0.7,
    "stop_sequences": [
      "\nUser:",
      "<|endoftext|>"
    ]
  },
  "theme": null,
  "logo_path": null,
  "show_system_prompt": true,
  "show_context": true
}
API Key available: Yes


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()